In [2]:
import pandas as pd

In [3]:
df_po = pd.read_csv('data/full_data/Presence_only_occurrences/Presences_only_train.csv', sep=';')
df_po

,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
0,2774863,1.978418e+09,Jim Kingdon,iNaturalist RG,2018-09-29,272,2018,-10.53904,51.77186,2924908.0,3383275.0,4.0,8075,3018869,1000
1,1043549,2.397517e+09,Julie Hall,iNaturalist RG,2019-08-22,234,2019,-10.53875,51.77226,2924939.0,3383313.0,5.0,8075,3018870,1001
2,3104839,3.953467e+09,NaN,Pl@ntNet automatic,2019-07-10,191,2019,-10.47597,52.11013,2939549.0,3418442.0,5.0,6477,3018871,1002
3,1800638,2.294436e+09,gsauce,iNaturalist RG,2019-07-02,183,2019,-10.47555,52.15074,2940838.0,3422801.0,6.0,5783,3018872,1003
4,4873519,2.265857e+09,guillaumevigneault,iNaturalist RG,2019-06-07,158,2019,-10.47497,52.10978,2939604.0,3418385.0,30.0,9011,3018873,1004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4908314,2889852,3.892391e+09,Alenka Mihoric,iNaturalist RG,2018-05-09,129,2018,34.54567,35.64241,6523828.0,1753690.0,21.0,148,7000525,3980319
4908315,1261849,3.355773e+09,Alenka Mihoric,iNaturalist RG,2018-05-09,129,2018,34.54568,35.64243,6523828.0,1753692.0,38.0,625,7000526,3980320
4908316,844504,2.841228e+09,NaN,Observation.org,2019-01-24,24,2019,34.55360,35.64514,6524428.0,1754209.0,10.0,1088,7000527,3980321
4908317,346527,2.842044e+09,NaN,Observation.org,2019-01-24,24,2019,34.55388,35.64499,6524457.0,1754201.0,10.0,982,7000528,3980322


In [4]:
df_pa = pd.read_csv('data/full_data/Presence_Absence_surveys/Presences_Absences_train.csv', sep=';')
df_pa

,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
0,5140269,3.473385e+09,(IGN),Inventaire forestier IGN,2018-01-31,31,2018,7.300790,48.29273,4.120673e+06,2.801353e+06,1.0,6574,3018575,3980325
1,5279421,3.474159e+09,(IGN),Inventaire forestier IGN,2017-07-19,200,2017,1.483390,42.92599,3.625340e+06,2.241565e+06,1.0,6574,3011783,3980331
2,5058088,3.475680e+09,(IGN),Inventaire forestier IGN,2017-09-26,269,2017,6.739570,47.39633,4.074778e+06,2.703502e+06,1.0,6574,3018192,3980335
3,5146840,3.476162e+09,(IGN),Inventaire forestier IGN,2017-09-20,263,2017,4.587140,45.40615,3.897015e+06,2.492494e+06,1.0,6574,3015541,3980336
4,5255062,3.475124e+09,(IGN),Inventaire forestier IGN,2019-09-18,261,2019,6.955830,47.74829,4.092660e+06,2.741886e+06,1.0,6574,3018391,3980351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85320,5320539,NaN,NaN,CBNMed,1621461600,140,2021,5.094570,44.18660,3.928254e+06,2.354689e+06,10.0,9456,76900,3993145
85321,5392608,NaN,NaN,CBNMed,1497909600,171,2017,5.781197,43.74345,3.980545e+06,2.302273e+06,10.0,9456,94452,3986894
85322,5353058,NaN,NaN,CBNMed,1621461600,140,2021,5.094620,44.18680,3.928259e+06,2.354711e+06,10.0,9456,76905,3993923
85323,5102872,2.907311e+09,"Shoesmith, J. Judith",Nat. plant monitoring UK,2017-06-07,158,2017,0.785642,51.70740,3.686016e+06,3.217623e+06,7.1,6081,3010687,3990777


In [7]:
len(df_po.speciesId.unique())

10038

In [5]:
pa_species = df_pa.speciesId.unique()
len(pa_species)

2174

In [14]:
df_po_sample = df_po[df_po.speciesId.isin(pa_species)]
df_po_sample = df_po_sample.groupby('speciesId').sample(5, replace=True).drop_duplicates()
df_po_sample

,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
1265334,4411607,2.236843e+09,cochran,iNaturalist RG,2019-04-09,99,2019,3.795753,50.97396,3885826.0,3114318.0,5.00,1,4230365,1209733
684603,841990,2.975382e+09,NaN,Pl@ntNet automatic,2020-07-17,199,2020,-0.277882,51.67187,3612703.0,3223502.0,38.06,1,3663912,646250
203841,3874025,2.645112e+09,NaN,Pl@ntNet automatic,2019-05-12,132,2019,-3.758217,40.42640,3155011.0,2031736.0,3.00,1,3216620,198869
2584877,4789990,2.573985e+09,Pedro Hafermann,iNaturalist RG,2020-02-15,46,2020,8.703934,50.63101,4229301.0,3058507.0,5.00,1,5392251,2367754
3058241,4335138,2.895769e+09,Miljøstyrelsen,DEN Environmental Portal,2019-06-18,169,2019,9.771029,55.28427,4306444.0,3575495.0,3.00,1,5660238,2636267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3547979,245836,3.949759e+09,NaN,Pl@ntNet observations,2021-06-21,172,2021,10.811430,45.93154,4384013.0,2535753.0,13.00,10039,5934713,2911918
524956,3636591,3.953747e+09,NaN,Pl@ntNet automatic,2021-10-07,280,2021,-1.399922,53.06815,3560433.0,3388745.0,7.54,10039,3514238,496543
193016,3081319,1.933545e+09,summerhenderson,iNaturalist RG,2018-10-11,284,2018,-3.880850,57.57739,3495567.0,3911084.0,5.00,10039,3206160,188420
4122816,944837,3.949039e+09,NaN,Pl@ntNet observations,2020-10-19,293,2020,12.556730,41.85038,4534071.0,2086647.0,14.19,10039,6341063,3318887


In [15]:
df_po_sample.to_csv('data/full_data/Presence_only_occurrences/Presences_only_train_2k_species.csv',sep=';')

In [19]:
df_pa_sample = df_pa.groupby('speciesId').sample(5, replace=True).drop_duplicates()
df_pa_sample


,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
82489,5341950,NaN,NaN,CBNMed,1621202400,137,2021,6.499306,43.272340,4.036152e+06,2.247126e+06,5.0,1,117034,3989331
82488,5463927,NaN,NaN,CBNMed,1490565600,86,2017,5.388190,43.277980,3.945891e+06,2.252630e+06,0.0,1,84011,4004892
82490,5425411,NaN,NaN,CBNMed,1491170400,93,2017,5.394488,43.303300,3.946564e+06,2.255400e+06,0.0,1,84134,4001100
50644,5304468,3.474198e+09,(IGN),Inventaire forestier IGN,2017-02-15,46,2017,6.769430,47.755530,4.078722e+06,2.743283e+06,1.0,5,3018224,3985097
50638,5017107,2.907389e+09,"Gibbs, V. Violet",Nat. plant monitoring UK,2017-05-29,149,2017,1.134387,51.300329,3.704429e+06,3.169633e+06,7.1,5,3011283,3985441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59229,5353754,NaN,NaN,CBNMed,1566856800,239,2019,7.090280,43.858200,4.086579e+06,2.309963e+06,10.0,10039,128338,3982256
59241,5636728,NaN,NaN,CBNMed,1634853600,295,2021,5.021850,43.466200,3.917442e+06,2.275354e+06,10.0,10039,73634,4001453
59253,5665172,NaN,NaN,CBNMed,1625004000,181,2021,6.499038,43.272350,4.036131e+06,2.247128e+06,5.0,10039,117028,3988696
59234,5666629,NaN,NaN,CBNMed,1590357600,146,2020,6.549660,43.188680,4.039847e+06,2.237675e+06,10.0,10039,118010,3982028


In [20]:
df_pa_sample.to_csv('data/full_data/Presence_Absence_surveys/Presences_Absences_train_2k_species.csv',sep=';')